In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [3]:
trainDf = pd.read_csv('train.csv')
trainDf.shape

(12989, 43)

In [4]:
trainDf = trainDf.loc[:, trainDf.nunique() != 1]
corr = trainDf.corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

,Id,duration,src_bytes,dst_bytes,hot,logged_in,num_compromised,num_root,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
Id,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
duration,0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
src_bytes,-0.02,-0.03,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
dst_bytes,-0.00,-0.02,0.60,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
hot,-0.02,-0.03,0.99,0.60,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
logged_in,-0.01,-0.00,0.82,0.62,0.81,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
num_compromised,-0.02,-0.03,0.99,0.59,0.99,0.80,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
num_root,-0.00,-0.00,-0.00,-0.00,-0.00,0.03,0.04,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
num_shells,0.01,-0.00,-0.00,-0.00,-0.00,0.02,-0.00,-0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
num_access_files,0.01,-0.00,-0.00,0.00,-0.00,0.02,-0.00,-0.00,-0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [5]:
Id = trainDf.pop('Id')
toa = trainDf.pop('type_of_attack')

total_columns = list(trainDf.columns)
number_columns = list(trainDf.select_dtypes('number').columns)
print('number of columns with numbers: ',number_columns)
print('total columns: ',total_columns)

trainDf_X = trainDf[number_columns]

total_columns, number_columns = set(total_columns), set(number_columns)
nan_columns = total_columns - number_columns
print(nan_columns)

protocolType = np.array(trainDf['protocol_type'])
flag  = np.array(trainDf['flag'])
service = np.array(trainDf['service'])



number of columns with numbers:  ['duration', 'src_bytes', 'dst_bytes', 'hot', 'logged_in', 'num_compromised', 'num_root', 'num_shells', 'num_access_files', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']
total columns:  ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'hot', 'logged_in', 'num_compromised', 'num_root', 'num_shells', 'num_access_files', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_r

In [6]:
protocolType = np.reshape(protocolType, (protocolType.shape[0], 1))
flag = np.reshape(flag, (flag.shape[0], 1))
service = np.reshape(service, (service.shape[0], 1))

labels = toa.nunique()

In [7]:
toa = np.array(toa)
toa = np.reshape(toa, (toa.shape[0],1))

In [8]:
label_encoder = preprocessing.LabelEncoder()

encoded_toa = label_encoder.fit_transform(toa)
encoded_protocolType = label_encoder.fit_transform(protocolType)
encoded_flag = label_encoder.fit_transform(flag)
encoded_service = label_encoder.fit_transform(service)

C:\Users\akish\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [9]:

np.unique(encoded_toa), np.unique(encoded_protocolType), np.unique(encoded_flag), np.unique(encoded_service)

(array([0, 1, 2, 3, 4]),
 array([0, 1, 2]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60]))

In [10]:
enc = OneHotEncoder(handle_unknown='ignore')
encoded_toa = np.reshape(encoded_toa, (encoded_toa.shape[0],1))
Y = enc.fit_transform(encoded_toa)

In [11]:
encoded_protocolType = np.reshape(encoded_protocolType, (encoded_protocolType.shape[0],1))
encoded_service = np.reshape(encoded_service, (encoded_service.shape[0],1))
encoded_flag = np.reshape(encoded_flag, (encoded_flag.shape[0],1))

In [12]:
protocolType = enc.fit_transform(encoded_protocolType)
service = enc.fit_transform(encoded_service)
flag = enc.fit_transform(encoded_flag)

In [13]:
protocolType.shape, service.shape, flag.shape

((12989, 3), (12989, 61), (12989, 9))

In [14]:
norm = MinMaxScaler()

# transform training data
X_train = norm.fit_transform(trainDf_X)

In [15]:
pt = protocolType.toarray()
f = flag.toarray()
s = service.toarray()

In [16]:
pt_f = np.concatenate((pt, f), axis = 1)
x = np.concatenate((pt_f, s), axis = 1)
X = np.concatenate((x, X_train), axis = 1)

In [17]:
Y = Y.toarray()

In [18]:
X.shape, Y.shape

((12989, 101), (12989, 5))

In [19]:
from sklearn.utils import shuffle

X,Y = shuffle(X,Y)

In [20]:
model = Sequential([
    Dense(units=128, input_shape=(1,101), activation='relu'),
    Dense(units=256, activation='relu'),
    Dense(units=5, activation='softmax')
])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 128)            13056     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 256)            33024     
_________________________________________________________________
dense_2 (Dense)              (None, 1, 5)              1285      
Total params: 47,365
Trainable params: 47,365
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
history = model.fit(x=X, y=Y, validation_split=0.33,  batch_size=1024, epochs=50, verbose=2)

Epoch 1/50
9/9 - 1s - loss: 1.2157 - accuracy: 0.7798 - val_loss: 0.7760 - val_accuracy: 0.9277
Epoch 2/50
9/9 - 0s - loss: 0.5773 - accuracy: 0.9315 - val_loss: 0.3512 - val_accuracy: 0.9419
Epoch 3/50
9/9 - 0s - loss: 0.2676 - accuracy: 0.9406 - val_loss: 0.1665 - val_accuracy: 0.9498
Epoch 4/50
9/9 - 0s - loss: 0.1242 - accuracy: 0.9739 - val_loss: 0.0765 - val_accuracy: 0.9951
Epoch 5/50
9/9 - 0s - loss: 0.0585 - accuracy: 0.9959 - val_loss: 0.0405 - val_accuracy: 0.9951
Epoch 6/50
9/9 - 0s - loss: 0.0308 - accuracy: 0.9961 - val_loss: 0.0247 - val_accuracy: 0.9951
Epoch 7/50
9/9 - 0s - loss: 0.0187 - accuracy: 0.9962 - val_loss: 0.0175 - val_accuracy: 0.9951
Epoch 8/50
9/9 - 0s - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.0139 - val_accuracy: 0.9953
Epoch 9/50
9/9 - 0s - loss: 0.0095 - accuracy: 0.9985 - val_loss: 0.0116 - val_accuracy: 0.9972
Epoch 10/50
9/9 - 0s - loss: 0.0075 - accuracy: 0.9997 - val_loss: 0.0102 - val_accuracy: 0.9974
Epoch 11/50
9/9 - 0s - loss: 0.0061 - a

In [24]:
model.save('NAPmodel')

INFO:tensorflow:Assets written to: NAPmodel\assets


In [25]:
from keras.models import load_model
Model = load_model('NAPmodel')

In [26]:
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 128)            13056     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 256)            33024     
_________________________________________________________________
dense_2 (Dense)              (None, 1, 5)              1285      
Total params: 47,365
Trainable params: 47,365
Non-trainable params: 0
_________________________________________________________________


In [27]:
_, train_acc = model.evaluate(X, Y, verbose=0)
#_, test_acc = model.evaluate(testX, testy, verbose=0)
print('Train: %.3f' % (train_acc))
# plot loss during training

Train: 1.000


In [29]:
from sklearn.metrics import classification_report

In [33]:
y_pred = model.predict(X, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

Y
y_pred_bool.shape

406/406 [==============================] - 0s 516us/step


(12989,)

In [35]:
Y_check = np.argmax(Y, axis = 1)

In [36]:
Y_check

array([1, 4, 1, ..., 1, 3, 0], dtype=int64)

In [37]:
y_pred_bool

array([1, 4, 1, ..., 1, 3, 0], dtype=int64)

In [39]:
y_pred = model.predict(X, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(Y_check, y_pred_bool))

203/203 [==============================] - 0s 636us/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1400
           1       1.00      1.00      1.00      5318
           2       1.00      1.00      1.00       561
           3       1.00      1.00      1.00      1999
           4       1.00      1.00      1.00      3711

    accuracy                           1.00     12989
   macro avg       1.00      1.00      1.00     12989
weighted avg       1.00      1.00      1.00     12989

